In [4]:
from PIL import Image
import os
import zipfile

def convert_and_resize(image_path, output_dir):
    image = Image.open(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    resized_image = image.resize((512, 512))
    output_path = os.path.join(output_dir, f"{base_name}.png")
    resized_image.save(output_path, format="PNG")
    return base_name, image, output_path

def generate_tiles(full_image, row, col, layer, output_dir):
    tile_size = 512
    scale = 3 ** layer
    grid_size = scale
    full_crop_size = tile_size * grid_size
    resized_image = full_image.resize((full_crop_size, full_crop_size))

    tiles = []
    for i in range(grid_size):
        for j in range(grid_size):
            left = j * tile_size
            upper = i * tile_size
            right = left + tile_size
            lower = upper + tile_size
            tile = resized_image.crop((left, upper, right, lower))
            global_row = row * grid_size + i
            global_col = col * grid_size + j
            filename = f"{global_row}_{global_col}.png"
            tile_path = os.path.join(output_dir, filename)
            tile.save(tile_path)
            tiles.append(tile_path)
    return tiles

def zip_tiles(tile_paths, zip_name):
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        for path in tile_paths:
            arcname = os.path.basename(path)
            zipf.write(path, arcname=arcname)
    return zip_name

def infer_coordinates_from_filename(filename):
    base = os.path.splitext(os.path.basename(filename))[0]
    parts = base.split("_")
    if len(parts) != 2 or not all(part.isdigit() for part in parts):
        raise ValueError("Filename must be in the format row_col.ext")
    return int(parts[0]), int(parts[1])

# === Example usage ===
# Update these paths accordingly:
#image_path = "./originals/0_0.png"  # Input image filename
image_path = "C:/Users/brian/Documents/Tiled Images/thangka/originals/4_13.png"
output_base_dir = "C:/Users/brian/Documents/Tiled Images/thangka/output"
os.makedirs(output_base_dir, exist_ok=True)

# Step 1: Convert & Resize
row, col = infer_coordinates_from_filename(image_path)
layer = 1  # Layer of the input image
base_name, full_image, resized_path = convert_and_resize(image_path, output_base_dir)
print(f"Resized image saved to: {resized_path}")

# Step 2: Generate tiles
if layer <= 2:
    tiles_layer_3 = generate_tiles(full_image, row, col, 2, os.path.join(output_base_dir, "Layer+1"))
    #zip_tiles(tiles_layer_3, os.path.join(output_base_dir, "layer_3.zip"))

if layer == 1:
    tiles_layer_2 = generate_tiles(full_image, row, col, 1, os.path.join(output_base_dir, "Layer+2"))
    #zip_tiles(tiles_layer_2, os.path.join(output_base_dir, "layer_2.zip"))

print("Tile generation complete.")


Resized image saved to: C:/Users/brian/Documents/Tiled Images/thangka/output\4_13.png
Tile generation complete.


In [3]:
from PIL import Image
import os

# ==== CONFIGURATION ====
max_layers = 5
input_image_path = r"C:\Users\brian\Documents\Tiled Images\thangka\originals\layer_2\7_4.png"
output_base_path = r"C:\Users\brian\Documents\Tiled Images\thangka\python output"

tile_size = 512
scale_factor = 3  # Each layer increases by 3x per axis


def infer_coordinates_from_filename(filename):
    """Extract row and col from a filename like 6_7.png"""
    base = os.path.splitext(os.path.basename(filename))[0]
    row, col = map(int, base.split("_"))
    return row, col


def ensure_dir(path):
    os.makedirs(path, exist_ok=True)


def crop_tile(original_img, x_index, y_index, crop_size, final_layer):
    """Crop from original image and resize to 512x512"""
    left = x_index * crop_size
    upper = y_index * crop_size
    right = left + crop_size
    lower = upper + crop_size

    tile = original_img.crop((left, upper, right, lower)).resize((tile_size, tile_size), Image.LANCZOS)
    return tile


def generate_all_layers(original_img, start_row, start_col, starting_layer):
    """Recursively generate tile grids from original image through max_layers"""

    for layer in range(starting_layer, max_layers + 1):
        depth = layer - starting_layer
        grid_size = scale_factor ** depth
        crop_size = original_img.width // grid_size

        output_dir = os.path.join(output_base_path, f"layer_{layer}")
        ensure_dir(output_dir)

        for i in range(grid_size):
            for j in range(grid_size):
                global_row = start_row * grid_size + i
                global_col = start_col * grid_size + j

                tile = crop_tile(original_img, j, i, crop_size, layer)
                filename = f"{global_row}_{global_col}.png"
                tile.save(os.path.join(output_dir, filename))


def main():
    # Step 1: Load original image and get row/col
    original_img = Image.open(input_image_path)
    row, col = infer_coordinates_from_filename(input_image_path)

    # Step 2: Save resized 512×512 version to its layer_{N} folder
    starting_layer = int(os.path.basename(os.path.dirname(input_image_path)).split("_")[1])
    resized = original_img.resize((tile_size, tile_size), Image.LANCZOS)
    layer_dir = os.path.join(output_base_path, f"layer_{starting_layer}")
    ensure_dir(layer_dir)
    resized.save(os.path.join(layer_dir, f"{row}_{col}.png"))

    # Step 3: Generate child layers recursively from original image
    generate_all_layers(original_img, row, col, starting_layer)

    print(f"Finished processing {input_image_path} through layer {max_layers}.")


if __name__ == "__main__":
    main()


Finished processing C:\Users\brian\Documents\Tiled Images\thangka\originals\layer_2\7_4.png through layer 5.


In [43]:
from PIL import Image
import os
import shutil
import re

# === CONFIGURATION ===
max_layers = 5
originals_root = r"C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals"
output_root = r"C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles"
tile_size = 512
scale_factor = 3


def infer_coordinates_from_filename(filename):
    base = os.path.splitext(os.path.basename(filename))[0]
    row, col = map(int, base.split("_"))
    return row, col


def ensure_dir(path):
    os.makedirs(path, exist_ok=True)


def crop_tile(original_img, x_index, y_index, crop_size):
    left = x_index * crop_size
    upper = y_index * crop_size
    right = left + crop_size
    lower = upper + crop_size

    tile = original_img.crop((left, upper, right, lower)).resize((tile_size, tile_size), Image.LANCZOS)
    return tile


def generate_tiles(original_img, start_row, start_col, input_layer):
    """
    Generate tiles starting from layer input_layer + 1 up to max_layers.
    Tile generation is always based on the original image.
    """
    max_layer_for_this_image = max(input_layer + 2, max_layers)
    for layer in range(input_layer + 1, max_layer_for_this_image + 1):
        depth = layer - input_layer  # relative depth
        grid_size = scale_factor ** depth
        crop_size = original_img.width // grid_size
        output_dir = os.path.join(output_root, f"layer_{layer}")
        ensure_dir(output_dir)

        total_tiles = grid_size * grid_size
        print(f"  🔸 Generating {total_tiles} tiles for layer_{layer} ({grid_size}×{grid_size})")

        for i in range(grid_size):
            for j in range(grid_size):
                global_row = start_row * grid_size + i
                global_col = start_col * grid_size + j

                tile = crop_tile(original_img, j, i, crop_size)
                filename = f"{global_row}_{global_col}.png"
                tile.save(os.path.join(output_dir, filename))


def process_image(image_path, layer):
    try:
        print(f"\n🖼️ Processing {image_path} (layer_{layer})")
        original_img = Image.open(image_path)
        row, col = infer_coordinates_from_filename(image_path)

        # Step 1: Resize and save to its own layer
        resized = original_img.resize((tile_size, tile_size), Image.LANCZOS)
        layer_output = os.path.join(output_root, f"layer_{layer}")
        ensure_dir(layer_output)
        resized_path = os.path.join(layer_output, f"{row}_{col}.png")
        resized.save(resized_path)
        print(f"  ✅ Resized saved: {resized_path}")

        # Step 2: Tile generation only if layer < max
        if layer < max_layers or layer >= 4:
            generate_tiles(original_img, row, col, layer)
        else:
            print(f"  ⏹️ No tile generation for layer_{layer}")

        # Step 3: Archive
        archive_dir = os.path.join(os.path.dirname(image_path), "archive")
        ensure_dir(archive_dir)
        archived_path = os.path.join(archive_dir, os.path.basename(image_path))
        shutil.move(image_path, archived_path)
        print(f"  📦 Archived original: {archived_path}")

    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")


def extract_layer_number(folder_name):
    match = re.search(r'layer_(\d+)', folder_name)
    return int(match.group(1)) if match else None


def main():
    print(f"🔍 Scanning originals from layer_0 to layer_{max_layers}...\n")
    for layer_dir in sorted(os.listdir(originals_root)):
        layer_path = os.path.join(originals_root, layer_dir)
        if not os.path.isdir(layer_path):
            continue

        layer_num = extract_layer_number(layer_dir)
        if layer_num is None or layer_num > max_layers:
            continue

        archive_dir = os.path.join(layer_path, "archive")
        ensure_dir(archive_dir)

        for filename in os.listdir(layer_path):
            if not filename.lower().endswith(".jpg"):
                continue
            full_path = os.path.join(layer_path, filename)
            if os.path.exists(os.path.join(archive_dir, filename)):
                print(f"⏭️ Skipping archived file: {filename}")
                continue
            process_image(full_path, layer_num)

    print("\n✅ Batch processing complete.")


if __name__ == "__main__":
    main()


🔍 Scanning originals from layer_0 to layer_5...


🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_1\0_0.jpg (layer_1)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_1\0_0.png
  🔸 Generating 9 tiles for layer_2 (3×3)
  🔸 Generating 81 tiles for layer_3 (9×9)
  🔸 Generating 729 tiles for layer_4 (27×27)
  🔸 Generating 6561 tiles for layer_5 (81×81)
  📦 Archived original: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_1\archive\0_0.jpg

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_1\0_1.jpg (layer_1)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_1\0_1.png
  🔸 Generating 9 tiles for layer_2 (3×3)
  🔸 Generating 81 tiles for layer_3 (9×9)
  🔸 Generating 729 tiles for layer_4 (27×27)
  🔸 Generating 6561 tiles for layer_5 (81×81)
  📦 Archived

❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\1_2.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\1_4.jpg (layer_2)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_2\1_4.png
  🔸 Generating 9 tiles for layer_3 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\1_4.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\2_4.jpg (layer_2)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_2\2_4.png
  🔸 Generating 9 tiles for layer_3 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\2_4.jpg: [Errno 28] No space left on device



❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\5_1.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\5_2.jpg (layer_2)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_2\5_2.png
  🔸 Generating 9 tiles for layer_3 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\5_2.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\5_3.jpg (layer_2)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_2\5_3.png
  🔸 Generating 9 tiles for layer_3 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\5_3.jpg: [Errno 28] No space left on device



❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\8_0.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\8_1.jpg (layer_2)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_2\8_1.png
  🔸 Generating 9 tiles for layer_3 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\8_1.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\8_4.jpg (layer_2)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_2\8_4.png
  🔸 Generating 9 tiles for layer_3 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_2\8_4.jpg: [Errno 28] No space left on device



❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_3\22_14.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_3\22_16.jpg (layer_3)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_3\22_16.png
  🔸 Generating 9 tiles for layer_4 (3×3)
  🔸 Generating 81 tiles for layer_5 (9×9)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_3\22_16.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_3\26_23.jpg (layer_3)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_3\26_23.png
  🔸 Generating 9 tiles for layer_4 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals

  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_4\67_50.png
  🔸 Generating 9 tiles for layer_5 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_4\67_50.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_4\68_4.jpg (layer_4)
  ✅ Resized saved: C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\tiles\layer_4\68_4.png
  🔸 Generating 9 tiles for layer_5 (3×3)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_4\68_4.jpg: [Errno 28] No space left on device

🖼️ Processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_4\69_25.jpg (layer_4)
❌ Error processing C:\Users\brian\Documents\Tiled Images\stacked_dimensions_2_midjourney\originals\layer_4\69_25.jpg: [Errno 28] No space left 